In [1]:
! pip install pytube
! pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.1 MB/s eta 0:00:00


In [2]:
import re
import os
from pytube import YouTube
from pytube.exceptions import RegexMatchError, VideoUnavailable, PytubeError
from unidecode import unidecode

# This function is used to check if the given YouTube URL is valid.
def is_valid_youtube_url(url):
    # Regular expression to match YouTube URLs with or without "si" query parameter
    regex = r'^https?://(www\.)?youtu\.?be(\.com)?/[\w-]+(\?si=[\w-]+)?$'
    return re.match(regex, url) is not None

# This function is used to download video from a YouTube video with various resolutions.
def download_video(url):
    if not is_valid_youtube_url(url):
        print("Invalid YouTube URL.")
        return

    try:
        video = YouTube(url)  # Initialize a YouTube object with the given URL.
        streams = video.streams.filter(file_extension='mp4')

        unique_resolutions = set()  # To store unique resolutions.

        for stream in streams:
            if stream.resolution:
                unique_resolutions.add(stream.resolution)

        unique_resolutions = sorted(list(unique_resolutions), key=lambda x: int(x[:-1]))  # Sort resolutions numerically.

        if not unique_resolutions:
            print("No unique video resolutions found.")
            return
    except (RegexMatchError, VideoUnavailable, PytubeError) as e:
        print(f"Error while fetching video: {str(e)}")
        return

    print("Available video resolutions:")
    for i, resolution in enumerate(unique_resolutions):
        print(f"{i+1}. Resolution: {resolution}")

    choice = input("Enter the number of the resolution you want to download: ")
    try:
        choice = int(choice)
        if 1 <= choice <= len(unique_resolutions):
            selected_resolution = unique_resolutions[choice - 1]
            stream = streams.filter(resolution=selected_resolution).first()  # Select the video stream based on user choice.
            video_title = unidecode(video.title)  # Convert the video title to ASCII to avoid character issues.
            video_filename = f"{video_title}_{selected_resolution}.mp4"  # Create a video file name with the video title and resolution.

            try:
                stream.download(output_path=".", filename=video_filename)  # Download the video to the current directory with the appropriate name.
                print(f"{selected_resolution} video ({video_filename}) successfully downloaded.")
            except Exception as e:
                print(f"Error while downloading video: {str(e)}")
        else:
            print("Invalid choice. Please select a valid number.")
    except ValueError:
        print("Invalid choice. Please enter a valid number.")

if __name__ == "__main__":
    url = input("Enter the YouTube video URL: ")
    download_video(url)  # Call the download_video function to download video from the provided URL with the specified resolution.

Enter the YouTube video URL: https://youtu.be/okNIj3A48eg?si=FrSqH56Y2NLyedRD
Available video resolutions:
1. Resolution: 144p
2. Resolution: 240p
3. Resolution: 360p
4. Resolution: 480p
5. Resolution: 720p
6. Resolution: 1080p
Enter the number of the resolution you want to download: 6
1080p video (After Nourway - Never Mine (Lyrics) | It's a simple "Good morning""Good night" and "How are you?"_1080p.mp4) successfully downloaded.
